# **Welkom bij mijn analytics rapport**

# **Bibliotheken Importeren**

In deze stap importeer ik alle benodigde bibliotheken die gebruikt zullen worden tijdens de analyse.

In [ ]:
import psycopg2 as ps
import pandas as pd
import plotly.express as px
import isodate

# **Dataset laden**

Ik maak verbinding met de PostgreSQL-database en laad de gegevens van de tabel videostatistics_fact in een Pandas DataFrame.

In [ ]:
conn2=ps.connect(
    dbname="indatad_s1148232",
    user="s1148232",password= "s1148232",
    host="95.217.3.61",
    port="5432"
)

Hieronder selecteer ik alles van de tabel videostatistics_fact

In [ ]:
df = pd.read_sql("SELECT * FROM videostatistics_fact;", conn2)
df

C:\Users\ramea\AppData\Local\Temp\ipykernel_4588\3647044155.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,video_statistics_id,date_id,video_id,category_id,deploy_id,total_views,total_likes,total_comments,popularity_score,previous_week_views,previous_week_likes
0,1814,5277,nBN9zG1JNPg,28,15,158941,5200,133,Not Popular,155064,5086
1,1670,5277,nBN9zG1JNPg,28,14,158375,5183,132,Not Popular,154611,5071
2,1526,5277,nBN9zG1JNPg,28,13,157900,5167,132,Not Popular,154611,5071
3,1824,5390,RsXLT2z3X8g,28,15,55228,1948,107,Not Popular,50812,1830
4,1680,5390,RsXLT2z3X8g,28,14,54684,1939,107,Not Popular,50293,1813
...,...,...,...,...,...,...,...,...,...,...,...
913,1160,5282,mz_4QLvz2HM,28,10,35157,757,285,Not Popular,0,0
914,1446,5282,mz_4QLvz2HM,28,12,35337,761,286,Not Popular,34922,755
915,1447,5280,4FT5RYuifwE,28,12,101631,2117,77,Not Popular,100699,2106
916,1304,5280,4FT5RYuifwE,28,11,101581,2116,77,Not Popular,100699,2106


# **Wat zijn de top 5 meest gelikede videos?**

Deze SQL-query haalt de vijf meest gelikede video's op, samen met hun publicatiedatum. Vervolgens maak ik een staafdiagram om deze video's te visualiseren.

In [ ]:
query = """
    WITH LatestDeploy AS (
        SELECT
            vd.title,
            vsf.total_likes,
            vd.published_date,
            vsf.deploy_id,
            ROW_NUMBER() OVER (PARTITION BY vd.video_id ORDER BY vsf.deploy_id DESC) AS rn
        FROM
            video_dim vd
        JOIN
            VideoStatistics_fact vsf ON vd.video_id = vsf.video_id
        JOIN
            date_dim dd ON dd.date_id = vsf.date_id
    )
    SELECT
        title,
        total_likes,
        published_date
    FROM
        LatestDeploy
    WHERE
        rn = 1
    ORDER BY
        total_likes DESC
    LIMIT 5;
"""

# Execute the SQL query and load the result into a DataFrame
top_5_likes = pd.read_sql(query, conn2)

# Display the results
top_5_likes


C:\Users\ramea\AppData\Local\Temp\ipykernel_4588\2201776888.py:30: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,title,total_likes,published_date
0,How the US Is Destroying Young People’s Future...,308767,2024-05-02 15:29:49
1,Why Having Fun Is the Secret to a Healthier Li...,49129,2022-07-21 16:08:30
2,Why US Politics Is Broken — and How To Fix It ...,38353,2024-05-07 15:30:27
3,"A Palestinian and an Israeli, Face to Face | A...",35814,2024-04-17 16:14:03
4,What Is an AI Anyway? | Mustafa Suleyman | TED,31502,2024-04-22 17:24:11


In [ ]:
fig = px.bar(top_5_likes,
             x='title',
             y='total_likes',
             title='Top 5 Most Liked Videos',
             labels={'title': 'Video Title', 'total_likes': 'Total Likes'},
             text='total_likes')

fig.show()

# **Wat zijn de top 5 meest views?**
Deze query haalt de vijf video's met de meeste views op en toont hun titels en publicatiedatums. Ook hier gebruik ik een staafdiagram voor de visualisatie.

In [ ]:
query = """
    WITH LatestDeploy AS (
        SELECT
            vd.title,
            vsf.total_views,
            vd.published_date,
            vsf.deploy_id,
            ROW_NUMBER() OVER (PARTITION BY vd.video_id ORDER BY vsf.deploy_id DESC) AS rn
        FROM
            video_dim vd
        JOIN
            VideoStatistics_fact vsf ON vd.video_id = vsf.video_id
        JOIN
            date_dim dd ON dd.date_id = vsf.date_id
    )
    SELECT
        title,
        total_views,
        published_date
    FROM
        LatestDeploy
    WHERE
        rn = 1
    ORDER BY
        total_views DESC
    LIMIT 5;
"""

# Execute the SQL query and load the result into a DataFrame
top_5_views = pd.read_sql(query, conn2)

# Display the results
top_5_views


C:\Users\ramea\AppData\Local\Temp\ipykernel_4588\3133359737.py:30: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,title,total_views,published_date
0,How the US Is Destroying Young People’s Future...,6025781,2024-05-02 15:29:49
1,Why Having Fun Is the Secret to a Healthier Li...,2515632,2022-07-21 16:08:30
2,What Is an AI Anyway? | Mustafa Suleyman | TED,1727020,2024-04-22 17:24:11
3,"A Palestinian and an Israeli, Face to Face | A...",1445191,2024-04-17 16:14:03
4,The Problem With Being “Too Nice” at Work | Te...,1220173,2024-05-29 15:37:13


In [ ]:
fig = px.bar(top_5_views,
             x='title',
             y='total_views',
             title='Top 5 Most Viewed Videos',
             labels={'title': 'Video Title', 'total_views': 'Total views'},
             text='total_views')

fig.show()

# **Wat is het gemiddelde aantal views?**

In deze stap bereken ik het gemiddelde aantal views voor alle video's in de VideoStatistics_fact tabel en rond ik het resultaat af voor eenvoudiger weergave.

In [ ]:
query = """
    SELECT AVG(vsf.total_views) AS average_views
    FROM VideoStatistics_fact vsf;
"""

avg_views = pd.read_sql(query, conn2)
avg_views

C:\Users\ramea\AppData\Local\Temp\ipykernel_4588\1785960080.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,average_views
0,193515.604575


In [ ]:

average_views = avg_views['average_views'][0]
rounded_average_views = round(average_views)
print(f"Het gemiddelde aantal views is: {rounded_average_views}")

Het gemiddelde aantal views is: 193516


# **Wat is het gemiddelde aantal video's per week?**

Hier bereken ik het aantal video's dat per week wordt geüpload, opgedeeld naar jaar en weeknummer, en visualiseer ik het resultaat met een lijndiagram.

In [ ]:
query = """
    SELECT
        EXTRACT(YEAR FROM vd.published_date) AS year,
        EXTRACT(WEEK FROM vd.published_date) AS week,
        COUNT(vd.video_id) AS videos_per_week
    FROM
        video_dim vd
    GROUP BY
        EXTRACT(YEAR FROM vd.published_date),
        EXTRACT(WEEK FROM vd.published_date)
    ORDER BY
        year DESC, week DESC;
"""

videos_per_week_df = pd.read_sql(query, conn2)
videos_per_week_df

C:\Users\ramea\AppData\Local\Temp\ipykernel_4588\3362843876.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,week,videos_per_week
0,2024.0,44.0,1
1,2024.0,43.0,5
2,2024.0,42.0,8
3,2024.0,41.0,7
4,2024.0,40.0,7
5,2024.0,39.0,7
6,2024.0,38.0,5
7,2024.0,37.0,5
8,2024.0,36.0,6
9,2024.0,35.0,5


In [ ]:
videos_per_week_df['year_week'] = videos_per_week_df['year'].astype(str) + '-W' + videos_per_week_df['week'].astype(str)

fig = px.line(videos_per_week_df,
              x='year_week',
              y='videos_per_week',
              title='Number of Videos Uploaded Per Week',
              labels={'year_week': 'Year-Week', 'videos_per_week': 'Videos Uploaded'},
              markers=True)

fig.update_layout(xaxis_tickangle=-45)

fig.show()

# **Gemiddelde videoduur berekenen**
Ik haal de duur van elke video op en zet deze om van ISO 8601 naar seconden, bereken vervolgens de gemiddelde duur en zet het resultaat om naar minuten en seconden.

In [ ]:
query = """
    SELECT duration
    FROM video_dim
    WHERE duration IS NOT NULL;
"""

video_durations_df = pd.read_sql(query, conn2)
video_durations_df

C:\Users\ramea\AppData\Local\Temp\ipykernel_4588\1200404562.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,duration
0,PT19M56S
1,PT15M23S
2,PT12M42S
3,PT11M8S
4,PT9M40S
...,...
140,PT47S
141,PT13M19S
142,PT12M36S
143,PT11M43S


In [ ]:
def iso8601_to_seconds(duration_str):
    try:
        duration = isodate.parse_duration(duration_str)
        return duration.total_seconds()
    except:
        return None

video_durations_df['duration_in_seconds'] = video_durations_df['duration'].apply(iso8601_to_seconds)

average_duration_in_seconds = video_durations_df['duration_in_seconds'].mean()

average_minutes = int(average_duration_in_seconds // 60)
average_seconds = int(average_duration_in_seconds % 60)

print(f"De gemiddelde videolengte is: {average_minutes} minuten en {average_seconds} seconden.")


De gemiddelde videolengte is: 14 minuten en 20 seconden.
